In [ ]:
# i will try saving everything in one file such that i dont have to open files

In [1]:
from architectures import *
from dataset_numpy import *
from utils import *
from torch.utils.data import Subset
from torch.utils.data import DataLoader
from tqdm import tqdm
from torch.utils.tensorboard import SummaryWriter
import json
import h5py
from icecream import ic
import numpy as np

#getting the configurations from the config file
with open('working_config.json', 'r') as f:
    config = json.load(f)
    
testset = RandomSeqDataset(r"D:\studium_tests\small_original_dataset.npz",config["testset_key"])
trainset = RandomSeqDataset(r"D:\studium_tests\small_original_dataset.npz",config["trainset_key"])


# Create dataloaders from each subset
test_loader = DataLoader(testset,  # we want to load our dataset
                         shuffle=False,  # shuffle for training
                         batch_size=1,  # 1 sample at a time
                         num_workers=0,
                         collate_fn=stack_if_possible_collate_fn  # no background workers
                         )

training_loader = DataLoader(trainset,  # we want to load our dataset
                             shuffle=False,  # shuffle for training
                             batch_size=4,  # stack 4 samples to a minibatch
                             num_workers=0,
                             collate_fn=stack_if_possible_collate_fn  # 2 background workers
                             )

In [3]:
for data in training_loader:
    print(data[0].shape)
    break

torch.Size([4, 90, 90])


In [7]:
import torch.nn as nn
import torch
import numpy as np
import torch.nn.functional as F

class AE_CNN(nn.Module):
    def __init__(self, ae_input_dim = 90*90, ae_middle_dim = 80*80, ae_hidden_dim = 70*70,
                n_input_channels = 1, n_output_channels = 1, conv_kernel_size = 5, n_hidden_layers=5, n_hidden_kernels=32):
        super(AE_CNN, self).__init__()
        # encoder
        #self.encoder_linear1 = nn.Linear(ae_input_dim, ae_input_dim)
        #self.encoder_linear2 = nn.Linear(ae_input_dim, ae_input_dim)
        
        # convolution
        hidden_layers = []
        for _ in range(n_hidden_layers):
            # Add a CNN layer
            layer = nn.Conv2d(in_channels=n_input_channels,
                              out_channels=n_hidden_kernels,
                              kernel_size=conv_kernel_size)
            hidden_layers.append(layer)
            # Add relu activation module to list of modules
            hidden_layers.append(nn.ReLU())
            n_input_channels = n_hidden_kernels

        # deconvolution
        for _ in range(n_hidden_layers + 1):
            # Add a CNN layer
            layer = nn.ConvTranspose2d(in_channels=n_input_channels,
                                       out_channels=n_hidden_kernels,
                                       kernel_size=conv_kernel_size)
            hidden_layers.append(layer)
            # Add relu activation module to list of modules
            hidden_layers.append(nn.ReLU())
            n_input_channels = n_hidden_kernels

            
        self.hidden_layers = nn.Sequential(*hidden_layers)

        self.output_layer = nn.Conv2d(in_channels=n_input_channels,
                                      out_channels=n_output_channels,
                                      kernel_size=conv_kernel_size)

        
        # decoder
        self.decoder_linear1 = nn.Linear(ae_input_dim, ae_input_dim)
        self.decoder_linear2 = nn.Linear(ae_input_dim, ae_input_dim)

    def encoder(self, x):
        x = self.encoder_linear1(x)
        x = F.relu(x)
        x = self.encoder_linear2(x)
        x = F.relu(x)
        return x

    def decoder(self, x):
        x = self.decoder_linear1(x)
        x = F.relu(x)
        x = self.decoder_linear2(x)
        x = torch.sigmoid(x)
        return x

    def forward(self, x):
         # Apply hidden layers module
        x = self.hidden_layers(x)

        # Apply last layer (=output layer)
        x = self.output_layer(x)
        x = x.view(-1,1,90*90)
        x = self.decoder(x)
        x = x.view(-1,1,90,90)
        return x

net = AE_CNN()
torch.save(net, os.path.join(config["results_path"], "train.pt"))

In [8]:
net

AE_CNN(
  (hidden_layers): Sequential(
    (0): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1))
    (1): ReLU()
    (2): Conv2d(32, 32, kernel_size=(5, 5), stride=(1, 1))
    (3): ReLU()
    (4): Conv2d(32, 32, kernel_size=(5, 5), stride=(1, 1))
    (5): ReLU()
    (6): Conv2d(32, 32, kernel_size=(5, 5), stride=(1, 1))
    (7): ReLU()
    (8): Conv2d(32, 32, kernel_size=(5, 5), stride=(1, 1))
    (9): ReLU()
    (10): ConvTranspose2d(32, 32, kernel_size=(5, 5), stride=(1, 1))
    (11): ReLU()
    (12): ConvTranspose2d(32, 32, kernel_size=(5, 5), stride=(1, 1))
    (13): ReLU()
    (14): ConvTranspose2d(32, 32, kernel_size=(5, 5), stride=(1, 1))
    (15): ReLU()
    (16): ConvTranspose2d(32, 32, kernel_size=(5, 5), stride=(1, 1))
    (17): ReLU()
    (18): ConvTranspose2d(32, 32, kernel_size=(5, 5), stride=(1, 1))
    (19): ReLU()
    (20): ConvTranspose2d(32, 32, kernel_size=(5, 5), stride=(1, 1))
    (21): ReLU()
  )
  (output_layer): Conv2d(32, 1, kernel_size=(5, 5), stride=(1, 1))


net = AE_CNN()
torch.save(net, os.path.join(config["results_path"], "AE_CNN_model1.pt"))

In [10]:
# Create an instance of our NET
net = torch.load(os.path.join(config["results_path"], "train.pt"))

# GPU will be much faster here
device = torch.device(config["device"])
net.to(device=device)

# also try rmse and absolute error
loss_function = torch.nn.MSELoss(reduction="mean")

# Use a SGD optimizer
optimizer = torch.optim.SGD(net.parameters(), lr=1e-2)
#optimizer = torch.optim.Adam(net.parameters(), lr=config["learningrate"])

# Define a tensorboard summary writer that writes to directory "results_path/tensorboard"
writer = SummaryWriter(log_dir=os.path.join(config["results_path"], 'tensorboard'), flush_secs=1)

best_loss = np.float("inf")
# Optimize our dsnn model using SGD:
print("Continuing the training:")

# for update in tqdm(range(200)):
with tqdm(total=config["n_updates"]) as pbar:
    for update in range(config["n_updates"]):
        x = 0
        for data in training_loader:
            # Compute the output
            input_array,  original = data
            # ic(input_array.unsqueeze(1).type(torch.float32).shape)
            output = net(input_array.unsqueeze(1).type(torch.float32))

            # Compute the loss
            loss = loss_function(output, original.type(torch.float32).unsqueeze(1))
            # Compute the gradients
            loss.backward()
            # Preform the update
            optimizer.step()
            # Reset the accumulated gradients
            optimizer.zero_grad()
            x += 1
            if x == 100:
                break

        # automatic saving of the best model
        if update % 1 == 0:
            with torch.no_grad():
                y = 0
                sum_loss = 0
                for tdata in test_loader:
                    input_array, original = data
                    output = net(input_array.unsqueeze(1).type(torch.float32))
                    sum_loss += loss_function(output, original.type(torch.float32).unsqueeze(1))
                    y += 1
                    if y == 10:
                        ic(sum_loss)
                        break
                    
            #if sum_loss < best_loss:
            #  #  torch.save(net, os.path.join(config["results_path"], "autosave", f"loss_{sum_loss}.pt"))
            #    best_loss = loss

        # Tensorboard
        #if update % 1 == 0:
        #    # Add losse as scalars to tensorboard
        #    writer.add_scalar(tag="training/loss", scalar_value=sum_loss,
        #                      global_step=update)
#
        #    # Add images to Tensorboard
        #    writer.add_image(tag="training/output", img_tensor=output[0], global_step=update)
#
        #    writer.add_image(tag="training/input", img_tensor=(original.unsqueeze(1)[0]), global_step=update)

        torch.save(net, os.path.join(config["results_path"], "train.pt"))
        pbar.update(1)

print("training finished")
#torch.save(net, os.path.join(config["results_path"], "my_model_A.pt"))
print("model saved")

  0%|          | 0/100 [00:00<?, ?it/s]

Continuing the training:


ic| sum_loss: tensor(1.1460)
  3%|▎         | 3/100 [07:45<4:10:39, 155.05s/it]


KeyboardInterrupt: 

110 sec with cpu

note to myself:
activate tensorboard like this:

activate programing_in_python_2
tensorboard --logdir=C:\Users\fabio\Google_Drive\AI_Studium\programing_python\ass2\ex5\results\tensorboard

http://localhost:6006/


In [ ]:
# trying another structure

In [1]:
from architectures import *
from dataset_numpy import *
from utils import *
from torch.utils.data import Subset
from torch.utils.data import DataLoader
from tqdm import tqdm
from torch.utils.tensorboard import SummaryWriter
import json
import h5py
from icecream import ic
import numpy as np

#getting the configurations from the config file
with open('working_config.json', 'r') as f:
    config = json.load(f)
    
testset = RandomSeqDataset(r"D:\studium_tests\small_original_dataset2.npz",config["testset_key"])
trainset = RandomSeqDataset(r"D:\studium_tests\small_original_dataset2.npz",config["trainset_key"])


# Create dataloaders from each subset
test_loader = DataLoader(testset,  # we want to load our dataset
                         shuffle=False,  # shuffle for training
                         batch_size=1,  # 1 sample at a time
                         num_workers=0,
                         collate_fn=stack_if_possible_collate_fn  # no background workers
                         )

training_loader = DataLoader(trainset,  # we want to load our dataset
                             shuffle=False,  # shuffle for training
                             batch_size=4,  # stack 4 samples to a minibatch
                             num_workers=0,
                             collate_fn=stack_if_possible_collate_fn  # 2 background workers
                             )

import torch.nn as nn
import torch
import numpy as np
import torch.nn.functional as F

class AE_CNN(nn.Module):
    def __init__(self, ae_input_dim = 90*90, ae_middle_dim = 80*80, ae_hidden_dim = 70*70,
                n_input_channels = 1, n_output_channels = 1, conv_kernel_size = 5, n_hidden_layers=5, n_hidden_kernels=32):
        super(AE_CNN, self).__init__()
        # encoder
        #self.encoder_linear1 = nn.Linear(ae_input_dim, ae_input_dim)
        #self.encoder_linear2 = nn.Linear(ae_input_dim, ae_input_dim)
        
        # convolution
        hidden_layers = []
        for _ in range(n_hidden_layers):
            # Add a CNN layer
            layer = nn.Conv2d(in_channels=n_input_channels,
                              out_channels=n_hidden_kernels,
                              kernel_size=conv_kernel_size)
            hidden_layers.append(layer)
            # Add relu activation module to list of modules
            hidden_layers.append(nn.ReLU())
            n_input_channels = n_hidden_kernels

            
        self.hidden_layers = nn.Sequential(*hidden_layers)

        self.output_layer = nn.Conv2d(in_channels=n_input_channels,
                                      out_channels=n_output_channels,
                                      kernel_size=conv_kernel_size)

        
        # decoder
        self.decoder_linear1 = nn.Linear(66*66, 80*80)
        self.decoder_linear2 = nn.Linear(80*80, ae_input_dim)

    #def encoder(self, x):
    #    x = self.encoder_linear1(x)
    #    x = F.relu(x)
    #    x = self.encoder_linear2(x)
    #    x = F.relu(x)
    #    return x

    def decoder(self, x):
        x = self.decoder_linear1(x)
        x = F.relu(x)
        x = self.decoder_linear2(x)
        x = torch.sigmoid(x)
        return x

    def forward(self, x):
         # Apply hidden layers module
        x = self.hidden_layers(x)

        # Apply last layer (=output layer)
        x = self.output_layer(x)
        x = x.view(-1,1,66*66)
        x = self.decoder(x)
        x = x.view(-1,1,90,90)
        return x

net = AE_CNN()
#torch.save(net, os.path.join(config["results_path"], "without_deconvolution.pt"))

In [8]:
class CNN(nn.Module):
    def __init__(self, n_input_channels: int, n_hidden_layers: int,
                 n_hidden_kernels: int, n_output_channels: int):
        """CNN, consisting of `n_hidden_layers` linear layers, using relu
        activation function in the hidden CNN layers.

        Parameters
        ----------
        n_input_channels: int
            Number of features channels in input tensor
        n_hidden_layers: int
            Number of hidden layers
        n_hidden_kernels: int
            Number of kernels in each hidden layer
        n_output_channels: int
            Number of features in output tensor
        """
        super(CNN, self).__init__()

        # encoder
        hidden_layers = []
        for _ in range(n_hidden_layers):
            # Add a CNN layer
            layer = nn.Conv2d(in_channels=n_input_channels,
                              out_channels=n_hidden_kernels,
                              kernel_size=7)
            hidden_layers.append(layer)
            # Add relu activation module to list of modules
            hidden_layers.append(nn.ReLU())
            n_input_channels = n_hidden_kernels

        # decoder
        for _ in range(n_hidden_layers + 1):
            # Add a CNN layer
            layer = nn.ConvTranspose2d(in_channels=n_input_channels,
                                       out_channels=n_hidden_kernels,
                                       kernel_size=7)
            hidden_layers.append(layer)
            # Add relu activation module to list of modules
            hidden_layers.append(nn.ReLU())
            n_input_channels = n_hidden_kernels

        self.hidden_layers = nn.Sequential(*hidden_layers)

        self.output_layer = nn.Conv2d(in_channels=n_input_channels,
                                      out_channels=n_output_channels,
                                      kernel_size=7)
        self.linear1 = nn.Linear(90*90, 90*90)

    def forward(self, x):
        """Apply CNN to `x`

        Parameters
        ----------
        x: torch.tensor
            Input tensor of shape (n_samples, n_input_channels, x, y)

        Returns
        ----------
        torch.tensor
            Output tensor of shape (n_samples, n_output_channels, u, v)
        """
        # Apply hidden layers module
        hidden_features = self.hidden_layers(x)

        # Apply last layer (=output layer)
        output = self.output_layer(hidden_features)
        output = x.view(-1,1,90*90)
        output = self.linear1(output)
        output = torch.sigmoid(output)
        output = output.view(-1,1,90,90)
        return output

net = CNN(n_input_channels=config["network_config"]["n_input_channels"], n_hidden_layers=config["network_config"]["n_hidden_layers"], n_hidden_kernels=config["network_config"]["n_hidden_kernels"],
          n_output_channels=config["network_config"]["n_output_channels"])
torch.save(net, os.path.join(config["results_path"], "new_cnn.pt"))

In [9]:
net

CNN(
  (hidden_layers): Sequential(
    (0): Conv2d(1, 32, kernel_size=(7, 7), stride=(1, 1))
    (1): ReLU()
    (2): Conv2d(32, 32, kernel_size=(7, 7), stride=(1, 1))
    (3): ReLU()
    (4): Conv2d(32, 32, kernel_size=(7, 7), stride=(1, 1))
    (5): ReLU()
    (6): ConvTranspose2d(32, 32, kernel_size=(7, 7), stride=(1, 1))
    (7): ReLU()
    (8): ConvTranspose2d(32, 32, kernel_size=(7, 7), stride=(1, 1))
    (9): ReLU()
    (10): ConvTranspose2d(32, 32, kernel_size=(7, 7), stride=(1, 1))
    (11): ReLU()
    (12): ConvTranspose2d(32, 32, kernel_size=(7, 7), stride=(1, 1))
    (13): ReLU()
  )
  (output_layer): Conv2d(32, 1, kernel_size=(7, 7), stride=(1, 1))
  (linear1): Linear(in_features=8100, out_features=8100, bias=True)
)

In [ ]:
#claring some folders
for key in config["to_clear"]:
    clear_folder(config["to_clear"][key])

# Create an instance of our NET
net = torch.load(os.path.join(config["results_path"], "new_cnn.pt"))

# GPU will be much faster here
device = torch.device(config["device"])
net.to(device=device)

# also try rmse and absolute error
loss_function = torch.nn.MSELoss(reduction="mean")

# Use a SGD optimizer
optimizer = torch.optim.SGD(net.parameters(), lr=1e-5)
#optimizer = torch.optim.SGD(net.parameters(), lr=1e+1)
#optimizer = torch.optim.Adam(net.parameters(), lr=config["learningrate"])

# Define a tensorboard summary writer that writes to directory "results_path/tensorboard"
writer = SummaryWriter(log_dir=os.path.join(config["results_path"], 'tensorboard'), flush_secs=1)

best_loss = np.float("inf")
# Optimize our dsnn model using SGD:
print("Continuing the training:")

# for update in tqdm(range(200)):
with tqdm(total=config["n_updates"]) as pbar:
    for update in range(config["n_updates"]):
        x = 0
        for data in training_loader:
            # Compute the output
            input_array,  original = data
            # ic(input_array.unsqueeze(1).type(torch.float32).shape)
            output = net(input_array.unsqueeze(1).type(torch.float32))

            # Compute the loss
            loss = loss_function(output, original.type(torch.float32).unsqueeze(1))
            # Compute the gradients
            loss.backward()
            # Preform the update
            optimizer.step()
            # Reset the accumulated gradients
            optimizer.zero_grad()
            x += 1
            if x == 1000:
                break

        # automatic saving of the best model
        if update % 1 == 0:
            with torch.no_grad():
                y = 0
                sum_loss = 0
                for tdata in test_loader:
                    input_array, original = data
                    output = net(input_array.unsqueeze(1).type(torch.float32))
                    sum_loss += loss_function(output, original.type(torch.float32).unsqueeze(1))
                    y += 1
                    if y == 100:
                        ic(sum_loss)
                        break
                    
            if sum_loss < best_loss:
                torch.save(net, os.path.join(config["results_path"], "autosave", f"loss_{sum_loss}.pt"))
                best_loss = loss

        # Tensorboard
        if update % 1 == 0:
            # Add losse as scalars to tensorboard
            writer.add_scalar(tag="training/loss", scalar_value=sum_loss,
                              global_step=update)

            # Add images to Tensorboard
            writer.add_image(tag="training/output", img_tensor=output[2], global_step=update)

            writer.add_image(tag="training/input", img_tensor=(original.unsqueeze(1)[2]), global_step=update)

        torch.save(net, os.path.join(config["results_path"], "new_cnn.pt"))
        pbar.update(1)

print("training finished")
#torch.save(net, os.path.join(config["results_path"], "my_model_A.pt"))
print("model saved")

  0%|          | 0/100 [00:00<?, ?it/s]

Continuing the training:


ic| sum_loss: tensor(4.6243)
  1%|          | 1/100 [09:09<15:06:25, 549.35s/it]

In [22]:
net

AE_CNN(
  (encoder_linear1): Linear(in_features=8100, out_features=8100, bias=True)
  (encoder_linear2): Linear(in_features=8100, out_features=8100, bias=True)
  (hidden_layers): Sequential(
    (0): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1))
    (1): ReLU()
    (2): Conv2d(32, 32, kernel_size=(5, 5), stride=(1, 1))
    (3): ReLU()
    (4): Conv2d(32, 32, kernel_size=(5, 5), stride=(1, 1))
    (5): ReLU()
    (6): Conv2d(32, 32, kernel_size=(5, 5), stride=(1, 1))
    (7): ReLU()
    (8): Conv2d(32, 32, kernel_size=(5, 5), stride=(1, 1))
    (9): ReLU()
  )
  (output_layer): Conv2d(32, 1, kernel_size=(5, 5), stride=(1, 1))
  (decoder_linear1): Linear(in_features=4356, out_features=8100, bias=True)
  (decoder_linear2): Linear(in_features=8100, out_features=8100, bias=True)
)

In [45]:
len(training_loader)*11/10/60

142.46833333333333

In [38]:
for data in training_loader:
    input_array,  original = data
    output = net(input_array.unsqueeze(1).type(torch.float32))
    break

from PIL import Image
import numpy as np

def array_to_img(image):
    img = Image.fromarray(image,)
    img.show()

array_to_img(original[2].detach().numpy()*255)
array_to_img(input_array[2].detach().numpy()*255)
array_to_img(output[2][0].detach().numpy()*255)

In [14]:
original.shape

torch.Size([4, 90, 90])